In [1]:
import sys

sys.path.append("../")
from hamilton_neural_network import (
    TrainTestData,
    HamiltonianNeuralNetwork,
    LatentHamiltonianNeuralNetwork,
)
from hamilton_system import HamiltonianSystem
from pdf_models import NegLogIndepedentGaussians, NegLogThreeDimRosenbrock
import tensorflow as tf
import matplotlib.pyplot as plt
from no_u_turn.nuts import NoUTurnSampling

tf.random.set_seed(0)


2.15.0


In [2]:
U = NegLogThreeDimRosenbrock()
K = NegLogIndepedentGaussians(
    tf.constant([0.0, 0.0, 0.0]), tf.constant([1.0, 1.0, 1.0])
)
q0 = tf.constant([[0.0, 0.0, 0.0]])
p0 = tf.random.normal(q0.shape)
T = 100.0
leap_frog_per_unit = 80
num_samples = 40
num_train = int(0.9 * num_samples * leap_frog_per_unit * T)

In [3]:
train_test_data = TrainTestData(
    num_samples, T, leap_frog_per_unit, q0, p0, U=U, K=K
)
samples = train_test_data()
tf.io.write_file("../exps/demo2_train_test_data.txt", tf.io.serialize_tensor(samples))

Generating samples...
Finished generating samples.


In [ ]:
file = tf.io.read_file("../exps/demo2_train_test_data.txt")
train_test_data = tf.io.parse_tensor(file, out_type=tf.float32)
train_test_data = tf.random.shuffle(train_test_data)
train_data = train_test_data[:num_train, :]
test_data = train_test_data[num_train:, :]
hnn = HamiltonianNeuralNetwork(3, 64)
hnn.build(input_shape=(1, 6))
train_hist, test_hist = hnn.train(
    epochs=10000,
    batch_size=2000,
    learning_rate=1e-4,
    train_set=train_data,
    test_set=test_data,
    save_dir="../exps/demo2_hnn.weights.h5",
    print_every=1000,
)



Training started...
Epoch 1000: Train loss 0.03684674948453903, Test loss 0.04638343304395676.
Epoch 2000: Train loss 0.013606764376163483, Test loss 0.015473277308046818.
Epoch 3000: Train loss 0.004412055015563965, Test loss 0.004277967382222414.
Epoch 4000: Train loss 0.008113712072372437, Test loss 0.06397075206041336.
Epoch 5000: Train loss 0.004450698848813772, Test loss 0.007437895052134991.


In [ ]:
ax, fig = plt.subplots()
fig.plot(train_hist, label="train", color="red")
fig.plot(test_hist, label="test", color="blue")
fig.legend()
fig.set_yscale("log")
fig.set_xlabel("epochs (x100)")
fig.set_ylabel("loss")
plt.show()

In [ ]:
hnn = HamiltonianNeuralNetwork(3, 64)
hnn.build(input_shape=(1, 6))
hnn.load_weights("../exps/demo2_hnn.weights.h5")
U = NegLogThreeDimRosenbrock()
K = NegLogIndepedentGaussians(
    tf.constant([0.0, 0.0, 0.0]), tf.constant([1.0, 1.0, 1.0])
)
q0 = tf.constant([[1.0, 2.0, 4.0]])
p0 = tf.constant([[0.0, 0.0, 0.0]])
T = 40.0
leap_frog_per_unit = 40
num_samples = 40
n_steps = 256
original_hamiltonian = HamiltonianSystem(U=U, K=K)

hist_original = original_hamiltonian.symplectic_integrate(
    q0, p0, 1 / leap_frog_per_unit, n_steps
)
hist_hnn = hnn.symplectic_integrate(q0, p0, 1 / leap_frog_per_unit, n_steps)

fig, ax = plt.subplots(1, 3)
ax[0].plot(hist_original[:, 0])
ax[0].plot(hist_hnn[:, 0])
ax[1].plot(hist_original[:, 1])
ax[1].plot(hist_hnn[:, 1])
ax[2].plot(hist_original[:, 2])
ax[2].plot(hist_hnn[:, 2])
plt.show()

In [ ]:
q0 = tf.constant([[0.0, 2.0, 8.0]])
nuts = NoUTurnSampling(num_samples=50000, q0=q0, dt=0.025, hnn=hnn)
nuts()
q_hist = tf.concat(nuts.q_hist, axis=0)
plt.hist(q_hist.numpy()[5000:, 0].flatten(), bins=30, color="blue")
plt.show()

In [ ]:
plt.scatter(q_hist.numpy()[1000:, 1].flatten(), q_hist.numpy()[1000:, 2].flatten())
plt.show()

In [ ]:
file = tf.io.read_file("../exps/demo2_train_test_data.txt")
train_test_data = tf.io.parse_tensor(file, out_type=tf.float32)
train_data = train_test_data[:num_train, :]
test_data = train_test_data[num_train:, :]
lhnn = LatentHamiltonianNeuralNetwork(3, 64, 2)
lhnn.build(input_shape=(1, 2))
train_hist, test_hist = lhnn.train(10000, 1000, 4e-5, train_data, test_data)
lhnn.save_weights("../exps/demo2_lhnn.weights.h5")

In [ ]:
ax, fig = plt.subplots()
fig.plot(train_hist, label="train", color="red")
fig.plot(test_hist, label="test", color="blue")
fig.legend()
fig.set_yscale("log")
fig.set_xlabel("epochs (x100)")
fig.set_ylabel("loss")
plt.show()

In [ ]:
q0 = tf.constant([[0.0, 0.0, 0.0]])
nuts = NoUTurnSampling(num_samples=500, q0=q0, dt=0.05, lhnn=lhnn)
nuts()
q_hist = tf.concat(nuts.q_hist, axis=0)
plt.hist(q_hist.numpy()[1000:, 0].flatten(), bins=30, color="blue")
plt.show()